In [1]:
# Installation des dépendances
!pip install transformers torch accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import warnings
warnings.filterwarnings("ignore")
# HF login to access Gemma 3 1B which is Open Weight but a gated model (need to ack the terms before accessing)
# Login your HuggingFace Account
from huggingface_hub import notebook_login
notebook_login()
# Execute and paste your token

In [3]:
# Model Config
MODEL_NAME = "google/gemma-3-1b-it"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Used device: {device}")
print("Model Loading...")

# Model config
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None,
    load_in_8bit=True if device == "cuda" else False,  # Quantification pour économiser la mémoire
)

# Pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Model loading succeed!")

def generate_response(prompt, max_length=128, temperature=0.7, top_p=0.9):
    """
    Generating answer
    """
    # Encodage du prompt
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Génération de la réponse
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            num_return_sequences=1
        )

    # decoding answer
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response[len(prompt):].strip()
    return response

def chat_loop():
    """
    interactive chat loop for testing purposes
    """
    print("\n👾 Gemma3-1b-it")
    print("=" * 25)
    print("type 'quit', 'exit' or 'bye' to leave")
    print("type 'clear' to wipe history")
    print("=" * 25)

    conversation_history = []

    while True:
        try:
            # User input
            user_input = input("\n👽 You: ").strip()

            # Commandes spéciales
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("\n👋 Bye!")
                break

            if user_input.lower() == 'clear':
                conversation_history = []
                print("🧹 history wiped!")
                continue

            if not user_input:
                continue

            # Construction du prompt avec l'historique
            if conversation_history:
                context = "\n".join(conversation_history[-6:])  # Garder les 6 derniers échanges
                prompt = f"{context}\nUser: {user_input}\nAssistant:"
            else:
                prompt = f"User: {user_input}\nAssistant:"

            print("\n👾 Gemma3-1b-it:", end=" ", flush=True)

            # Génération de la réponse
            response = generate_response(
                prompt,
                max_length=1024,
                temperature=0.7,
                top_p=0.9
            )

            print(response)

            # Mise à jour de l'historique
            conversation_history.append(f"Utilisateur: {user_input}")
            conversation_history.append(f"Assistant: {response}")

        except KeyboardInterrupt:
            print("\n\n⚠️ interruption detected. type 'quit' to leave properly.")
            continue
        except Exception as e:
            print(f"\n❌ Error: {str(e)}")
            continue

# Fonction pour tester le modèle avec un prompt simple
def test_model():
    """
    Model Testing
    """
    test_prompt = "Hi, How can I help you? Feel free to ask me about what is Project Zero and how does it contribute to security? or How does Google ensure the security of its cloud services?"
    print(f"\n👾 Gemma3-1b-it: '{test_prompt}'")
    response = generate_response(test_prompt, max_length=100)
    print(f"👾 Answer: {response}")

# Lancement du test puis du chat
if __name__ == "__main__":
    test_model()
    chat_loop()

Used device: cuda
Model Loading...


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

...

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Model loading succeed!

👾 Gemma3-1b-it: 'Hi, How can I help you? Feel free to ask me about what is Project Zero and how does it contribute to security? or How does Google ensure the security of its cloud services?'
👾 Answer: Let's talk about it!

👾 Gemma3-1b-it
type 'quit', 'exit' or 'bye' to leave
type 'clear' to wipe history

👽 You: what is Project Zero and how does it contribute to security?

👾 Gemma3-1b-it: Project Zero is a comprehensive, multi-year initiative at Google to address the evolving cybersecurity landscape. It's a significant shift in how Google approaches security, moving away from a purely reactive approach to a proactive, risk-based approach.

Here’s a breakdown of what it entails:

*   **Focus on Understanding:** Project Zero starts with a deep understanding of how Google’s systems and data are used. This involves extensive research, vulnerability scanning, and analysis.
*   **Threat Intelligence:**  It heavily relies on threat intelligence – gathering information a